# Multi-temporal analysis of vegetation indices variability
Made by Tomasz Dąbrowa

### Importing necessary libraries

In [ ]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import json, requests
import datetime
import rasterio
import rasterio.mask
from shapely.geometry import mapping, shape, Polygon, MultiPolygon
from shapely.wkt import dumps
import pandas as pd
import geopandas as gpd
import fiona
import statistics
#from osgeo import gdal, osr

## Defining useful functions

Those functions are responsible for obtaining specified data from path and to make projections

In [ ]:
def getYear(link):
    link_divided = link.split('/')
    return link_divided[5]
def getMonth(link):
    link_divided = link.split('/')
    return link_divided[6]
def getDay(link):
    link_divided = link.split('/')
    return link_divided[7]
def getDate(link):
    link_divided = link.split('/')
    return link_divided[5] + '-' + link_divided[6] + '-' + link_divided[7]
def getTile(link):
    link_divided = link.split('/')
    last = link_divided[len(link_divided)-1]
    return last[0:6]
def getBand(link):
    link_divided = link.split('/')
    last = link_divided[len(link_divided)-1]
    return last[23:26]
# def transformToEPSG2180(path, id):
#     src_ds = gdal.Open(path)
#     src_wkt = src_ds.GetProjection()
#     src_srs = osr.SpatialReference()
#     src_srs.ImportFromWkt(src_wkt)
#     dst_srs = osr.SpatialReference()
#     dst_srs.ImportFromEPSG(2180)
#     output_raster = f"/tmp/{id}.tif"
#     gdal.Warp(output_raster, src_ds, dstSRS=dst_srs.ExportToWkt())

def printReport(links_df, dates, values):
    print('====================== PROCESSING REPORT =========================')
    if len(links_df) == len(dates):
        print("All sattelite images processed")
    elif len(dates) != len(values):
        print("ERROR -> dates are not equal to indices values")
    else:
        print("Not every sattelite image processed because of the cloud cover")
        print('Available: ', len(links_df),'Used: ',len(dates))
    print('==================================================================')

def getFileName(path_to_file):
    full_path = path_to_file.split('/')
    filename = full_path[len(full_path)-1]
    filename = filename.split('.')
    filename_without_extension = filename[0]
    return filename_without_extension

def coordinatesToRasterEPSG(crs, coordinates):
    coordinates_in_raster_epsg = coordinates.to_crs(crs)
    geometry_raster_EPSG = coordinates_in_raster_epsg['geometry'][0]
    if isinstance(geometry_raster_EPSG, MultiPolygon):
        location_shapely_raster_EPSG = geometry_raster_EPSG.geoms[0]
    else:
        location_shapely_raster_EPSG = geometry_raster_EPSG
    return location_shapely_raster_EPSG
    

### Specifying input parameters
Choose time period that interests you for the analysis purposes.
- Specify start and end date in format YYYY-MM-DD.
- Type path to your ESRI Shapefile in EPSG 2180 containing one polygon or multipolygon

DO NOT change resolution parameter unless you know what you are doing (may cause errors)

In [ ]:
start = str("2023-01-01")  
end = str("2023-12-31")
shapefile_path = 'workspace/dummy_epsg2180.shp'

resolution = '20m' # DO NOT CHANGE

### Loading vector data 
Vector data that you provided is now projected to EPSG 4326 in order to make it usable for url request.
If data contains multipolygon type, only the first polygon is considered.

In [ ]:
coordinates = gpd.read_file(shapefile_path)
coordinates_EPSG4326 = coordinates.to_crs('EPSG:4326')
geometry_EPSG4326 = coordinates_EPSG4326['geometry'][0]
if isinstance(geometry_EPSG4326, MultiPolygon):
    # Choosing the first object from MultiPolygon object
    location_shapely = geometry_EPSG4326.geoms[0]
    print(location_shapely)
else:
    # if the geometry is the Polygon type already
    location_shapely = geometry_EPSG4326
location = str(location_shapely)

### Sending query by ODATA 
Sending request in order to get path for directories that contain sattelite date in eodata catalog.
A response is sorted by data ascending in time.
If we have more than one image per day, links to additional images are deleted.

In [ ]:
url = f"https://datahub.creodias.eu/odata/v1/Products?$filter=((Attributes/OData.CSC.DoubleAttribute/any(i0:i0/Name eq 'cloudCover' and i0/Value le 20)) and (ContentDate/Start ge {start}T00:00:00.000Z and ContentDate/Start le {end}T23:59:59.999Z) and (Online eq true) and (OData.CSC.Intersects(Footprint=geography'SRID=4326;{location}')) and (((((((Attributes/OData.CSC.StringAttribute/any(i0:i0/Name eq 'productType' and i0/Value eq 'S2MSI2A')))) and (Collection/Name eq 'SENTINEL-2'))))))&$expand=Attributes&$top=1000"
products = json.loads(requests.get(url).text)
tmp_cloud_cover = []    
path_list = []                           
for item in products['value']:
    path_list.append(item['S3Path'])     #append each S3Path (path) to the list
    #print(item['S3Path'])        
    for attribute in item['Attributes']:
        if attribute['Name'] == 'cloudCover':
            tmp_cloud_cover.append(attribute['Value'])
            break

path_list_sorted = sorted(path_list, key=getDate)

path_list_sorted_cut = []
seen_dates = set()
for path in path_list_sorted:
    date = getDate(path)
    if date not in seen_dates:
        path_list_sorted_cut.append(path)
        seen_dates.add(date)
path_list_sorted_cut


### Creating the Table of Links
A Pandas Dataframe is created that store information about sattelite data in format that is described below:
- directory_path: absolute path to .SAFE directory
- date: date of data
- tile_id: id of tile
- cloud_cover: parameter describing cloud coverage in %
- RED: path to red band (B04)
- NIR: path to near-infrared band (B8A)
- SCL: path to scene classification layer
- B05: path to B05 band
- B11: path to B11 band

In [ ]:
links = []
i_for_cloud_cover = 0
for directory_path in path_list_sorted_cut:
    
    matching_directories = glob.glob(os.path.join(directory_path, f"GRANULE/*/IMG_DATA/R{resolution}"))

    for directory in matching_directories:
        for root, dirs, files in os.walk(directory):
            for file in files:
                tmp_band = getBand(os.path.join(root, file))
                if tmp_band == 'B04':
                    tmp_red_path = os.path.join(root, file)
                if tmp_band == 'B8A':
                    tmp_nir_path = os.path.join(root, file)
                if tmp_band == 'SCL':
                    tmp_scl_path = os.path.join(root, file)
                if tmp_band == 'B05':
                    tmp_b05_path = os.path.join(root, file)
                if tmp_band == 'B11':
                    tmp_b11_path = os.path.join(root, file)
                
            links.append([directory_path, 
                          getDate(os.path.join(root, file)), 
                          getTile(os.path.join(root, file)), 
                          tmp_cloud_cover[i_for_cloud_cover], 
                          tmp_red_path, 
                          tmp_nir_path, 
                          tmp_scl_path,
                          tmp_b05_path,
                          tmp_b11_path])
            
    i_for_cloud_cover = i_for_cloud_cover + 1

    """
    links is an output. It is a list of lists. Each list has a structure that is described below:
    [directory path (absolute path ending on .SAFE catalog), date of image, Tile ID, Cloud cover (in %), path to red band, path to nir band, path to SCL band, path to B05, path to B11 band]
    """
links_df = pd.DataFrame(links, columns=['directory_path', 'date', 'tile_id', 'cloud_cover', 'RED', 'NIR', 'SCL', 'B05', 'B11'])

Overview of database with paths to data

In [ ]:
links_df.head()

## Indices computing
That is the main part of code. Iterating by date, all required indices are computed. Firstly, scene classification layer is opened and projected to EPSG 2180. If the mode indicates clouds or snow, the next image starts to be processed. If not, other bands are projected and indices are computed.
At the end, the report is printed.

It may take a while.

In [ ]:
dates = []
ndvi_means = []
ndre_means = []
savi_means = []
ndii_means = []
evi2_means = []

for i in range(len(links_df)):

    print(round(i/len(links)*100,0),'%','    Processing date: ', links_df['date'][i])

    with rasterio.open(links_df['SCL'][i]) as scl_canal_src:
        location_shapely_raster_EPSG = coordinatesToRasterEPSG(scl_canal_src.crs, coordinates)
        scl_canal, scl_canal_transform = rasterio.mask.mask(scl_canal_src, [location_shapely_raster_EPSG], crop=True)
        scl_canal_meta = scl_canal_src.meta
        scl_canal_meta.update({"driver": "GTiff",
                    "height": scl_canal.shape[1],
                    "width": scl_canal.shape[2],
                    "transform": scl_canal_transform})

        scl_canal_flattened = scl_canal.flatten().astype(float)
        scl_canal_flattened[scl_canal_flattened == 0] = np.nan
        scl_mode = statistics.mode(scl_canal_flattened)

        # checking if there is not a high cloud coverage or snow
        if scl_mode not in [3, 8, 9, 11]:
            
            with rasterio.open(links_df['RED'][i]) as red_canal_src, rasterio.open(links_df['NIR'][i]) as nir_canal_src, rasterio.open(links_df['B05'][i]) as b05_canal_src, rasterio.open(links_df['B11'][i]) as b11_canal_src:
                # other bands clipping
                #RED
                red_canal, red_canal_transform = rasterio.mask.mask(red_canal_src, [location_shapely_raster_EPSG], crop=True)
                red_canal_meta = red_canal_src.meta
                red_canal_meta.update({"driver": "GTiff",
                            "height": red_canal.shape[1],
                            "width": red_canal.shape[2],
                            "transform": red_canal_transform})
                
                #NIR
                nir_canal, nir_canal_transform = rasterio.mask.mask(nir_canal_src, [location_shapely_raster_EPSG], crop=True)
                nir_canal_meta = nir_canal_src.meta
                nir_canal_meta.update({"driver": "GTiff",
                            "height": nir_canal.shape[1],
                            "width": nir_canal.shape[2],
                            "transform": nir_canal_transform})
                

                #B05
                b05_canal, b05_canal_transform = rasterio.mask.mask(b05_canal_src, [location_shapely_raster_EPSG], crop=True)
                b05_canal_meta = b05_canal_src.meta
                b05_canal_meta.update({"driver": "GTiff",
                            "height": b05_canal.shape[1],
                            "width": b05_canal.shape[2],
                            "transform": b05_canal_transform})
                #B11
                b11_canal, b11_canal_transform = rasterio.mask.mask(b11_canal_src, [location_shapely_raster_EPSG], crop=True)
                b11_canal_meta = b11_canal_src.meta
                b11_canal_meta.update({"driver": "GTiff",
                            "height": b11_canal.shape[1],
                            "width": b11_canal.shape[2],
                            "transform": b11_canal_transform})
                

                nan_mask = np.isnan(nir_canal[0])
                
                # NDVI computing (and dividing by zero errors disabling)
                with np.errstate(divide='ignore', invalid='ignore'):
                    ndvi = (nir_canal[0] - red_canal[0]) / (nir_canal[0] + red_canal[0])

                
                # masking the water
                ndvi[scl_canal[0] == 6] = np.nan

                # switching to NaN NDVI values that are not in the range
                ndvi[ndvi > 1] = np.nan

                # NDRE computing (and dividing by zero errors disabling)
                with np.errstate(divide='ignore', invalid='ignore'):
                    ndre = (nir_canal[0] - b05_canal[0]) / (nir_canal[0] + b05_canal[0])

                # water masking
                ndre[scl_canal[0] == 6] = np.nan

                # switching to NaN NDRE values that are not in the range
                ndre[ndre > 1] = np.nan

                # Computing SAVI
                
                #describing l parameter for SAVI index based on SCL band values

                if scl_mode == 4:
                    l = 0.1
                else:
                    l = 0.5

                savi = ((nir_canal[0] - red_canal[0]) / (nir_canal[0] + red_canal[0] + l)) * (1 + l)

                
                savi[nan_mask] = np.nan

                #water masking
                savi[scl_canal[0] == 6] = np.nan

                # switching to NaN SAVI values that are not in the range
                savi[savi > 1] = np.nan
                savi[savi == 0] = np.nan


                # Computing NDII
                with np.errstate(divide='ignore', invalid='ignore'):
                    ndii = (nir_canal[0] - b11_canal[0]) / (nir_canal[0] + b11_canal[0])

                ndii[ndii > 1] = np.nan


                # Computing EVI2
                evi2 = 2.5 * ( (nir_canal[0] - red_canal[0]) / (nir_canal[0] + (2.4*red_canal[0]) + 1))

                # NaN masking
                evi2[evi2 == 0] = np.nan
                #water masking
                evi2[scl_canal[0] == 6] = np.nan
                #masking values upper than 2
                evi2[evi2 >= 2] = np.nan
                

                ndvi_mean = np.nanmean(ndvi)
                ndvi_means.append(ndvi_mean)

                ndre_mean = np.nanmean(ndre)
                ndre_means.append(ndre_mean)

                savi_mean = np.nanmean(savi)
                savi_means.append(savi_mean)

                ndii_mean = np.nanmean(ndii)
                ndii_means.append(ndii_mean)

                evi2_mean = np.nanmean(evi2)
                evi2_means.append(evi2_mean)

                dates.append(links_df['date'][i])
                
        else:
            print(f"Data from {links_df['date'][i]} not used")

print(100,'%')
printReport(links_df, dates, ndvi_means)


## Visualising the results
Results are ploted and saved to JPG file.

In [ ]:
filename_to_export = getFileName(shapefile_path) + f"_{start}_{end}"
dates_str = [datetime.datetime.strptime(date, "%Y-%m-%d") for date in dates]

fig, ax = plt.subplots()
ax.plot(dates_str, ndvi_means, 'go', label='NDVI')
ax.plot(dates_str, ndre_means, 'ro', label='NDRE')
ax.plot(dates_str, savi_means, 'yo', label='SAVI')
ax.plot(dates_str, ndii_means, 'bo', label='NDII')
ax.plot(dates_str, evi2_means, 'mo', label='EVI2')
ax.plot(dates_str, ndvi_means, 'g')
ax.plot(dates_str, ndre_means, 'r')
ax.plot(dates_str, savi_means, 'y')
ax.plot(dates_str, ndii_means, 'b')
ax.plot(dates_str, evi2_means, 'm')
plot_title = f"Mean indices values from {start} to {end} of {getFileName(shapefile_path)}"
ax.set_title(plot_title)
ax.set_ylabel('Index value')

# checking if there is a need for a logarythmic scale
if max(evi2_means) > 2:
    ax.set_yscale('log')


ax.legend()
plt.gcf().autofmt_xdate()

plt.savefig(filename_to_export + '.jpg')
plt.show()

### Creating time series for further analysis
Because of the cloud coverage and other factors, not all images can be used. Due to that fact, we do not have a regular timestep.
In order to make time series, lacking values must be interpolated.

The result is saved to CSV file.

In [ ]:
dates_tmp = pd.to_datetime(dates)
df = pd.DataFrame({'Date': dates_tmp, 'NDVI': ndvi_means, 'NDRE': ndre_means, 'SAVI': savi_means, 'NDII': ndii_means, 'EVI2': evi2_means})
date_steps = [(dates_str[i] - dates_str[i - 1]).days for i in range(1, len(dates_str))]
min_timestep = min(date_steps)
new_dates_df = pd.date_range(start=df['Date'].min(), end=df['Date'].max(), freq=f'{min_timestep}D')
new_ndvi_means_df = np.interp(
    x=new_dates_df.astype(np.int64) / 10**9,  # new dates in int format
    xp=df['Date'].astype(np.int64) / 10**9,   # original dates in int format
    fp=df['NDVI']                             # original index values
)
new_ndre_means_df = np.interp(
    x=new_dates_df.astype(np.int64) / 10**9,  
    xp=df['Date'].astype(np.int64) / 10**9,   
    fp=df['NDRE']                             
)
new_savi_means_df = np.interp(
    x=new_dates_df.astype(np.int64) / 10**9,  
    xp=df['Date'].astype(np.int64) / 10**9,   
    fp=df['SAVI']                             
)
new_ndii_means_df = np.interp(
    x=new_dates_df.astype(np.int64) / 10**9,  
    xp=df['Date'].astype(np.int64) / 10**9,   
    fp=df['NDII']                             
)
new_evi2_means_df = np.interp(
    x=new_dates_df.astype(np.int64) / 10**9,  
    xp=df['Date'].astype(np.int64) / 10**9,   
    fp=df['EVI2'] 
)
interpolated_df = pd.DataFrame({'Date': new_dates_df, 'NDVI': new_ndvi_means_df, 'NDRE': new_ndre_means_df, 'SAVI': new_savi_means_df, 'NDII': new_ndii_means_df, 'EVI2': new_evi2_means_df})

interpolated_df.to_csv(filename_to_export + '.csv', index=False)

Interpolated values

In [ ]:
interpolated_df.head()

### Map visualization of the last image

In [ ]:
print(f"Plots values dated to {links_df['date'][len(links_df)-1]}")

#plt.figure(figsize=(3,3))

# plt.imshow(scl_canal[0], cmap='gray')
# plt.colorbar(label='Pixel value')
# plt.show()

# plt.imshow(red_canal[0], cmap='gray')
# plt.colorbar(label='RED value')
# plt.show()

# plt.imshow(nir_canal[0], cmap='gray')
# plt.colorbar(label='NIR value')
# plt.show()
plt.figure(figsize=(12,12))
plt.subplot(3,3,1)
plt.imshow(ndvi, cmap="RdYlGn")
plt.colorbar()
plt.title('NDVI')
#plt.show()

plt.subplot(3,3,2)
plt.imshow(ndre, cmap="RdYlGn")
plt.colorbar()
plt.title('NDRE')
#plt.show()

plt.subplot(3,3,3)
plt.imshow(savi, cmap="RdYlGn")
plt.colorbar()
plt.title('SAVI')
#plt.show()

plt.subplot(3,3,4)
plt.imshow(ndii, cmap="RdYlGn")
plt.colorbar()
plt.title('NDII')
#plt.show()

plt.subplot(3,3,5)
plt.imshow(evi2, cmap="RdYlGn")
plt.colorbar()
plt.title('EVI2')

#plt.tight_layout()
plt.show()

# plt.imshow(nan_mask, cmap="gray")
# plt.colorbar(label='NaN Mask')
# plt.show()